In [ ]:
%run block_swipdg.ipynb

In [ ]:
import numpy as np

In [ ]:
print('estimating error ...')

local_sizes = [block_space.local_space(ii).size() for ii in range(block_space.num_blocks)]
local_starts = [int(np.sum(local_sizes[:ii])) for ii in range(block_space.num_blocks)]
local_starts.append(block_space.mapper.size)
localized_u_h_np = np.array(u_h_vector, copy=False)
localized_u_h_np = [localized_u_h_np[local_starts[ii]:local_starts[ii+1]] for ii in range(block_space.num_blocks)]
localized_u_h = [Vector(block_space.local_space(ii).size(), 0.) for ii in range(block_space.num_blocks)]
for ii in range(block_space.num_blocks):
    tmp = np.array(localized_u_h[ii], copy=False)
    tmp[:] = localized_u_h_np[ii][:]
local_boundary_info = make_subdomain_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.boundarysegmentindexbased',
                                                          'default': 'dirichlet',
                                                          'neumann': '[{} {}]'.format(inner_boundary_id,
                                                                                      inner_boundary_id+1)})

# the local indicators
from dune.gdt import (
    make_ESV2007_nonconformity_product_dd_subdomain_part_dd_subdomain_oversampled_part
        as make_local_nonconformity_product,
    make_ESV2007_residual_product_dd_subdomain_part_leaf_view as make_local_ESV2007_residual_product,
    make_OS2015_residual_product_dd_subdomain_part_leaf_view as make_local_OS2015_residual_product,
    make_ESV2007_diffusive_flux_product_dd_subdomain_part_leaf_view as make_local_diffusive_flux_product
)

eta_nc_squared = 0.
eta_r_ESV2007_squared = 0.
eta_r_OS2015_squared = 0.
eta_df_squared = 0.
for ii in range(grid.num_subdomains):
    neighborhood = grid.neighborhood_of(ii)
    neighborhood_space = block_space.restricted_to_neighborhood(neighborhood)
    restricted_u_h_vector = neighborhood_space.project_onto_neighborhood([localized_u_h[jj] for jj in neighborhood],
                                                                         neighborhood)
    restricted_u_h = make_discrete_function(neighborhood_space, restricted_u_h_vector, 'u_h')
    # eta_nc
    local_nonconformity_product = make_local_nonconformity_product(grid, ii, ii,
                                                                   local_boundary_info,
                                                                   lambda_, kappa,
                                                                   restricted_u_h, restricted_u_h,
                                                                   over_integrate=2)
    eta_nc_squared += local_nonconformity_product.apply2()
    # eta_r
    # The following is not optimal, but the pdelab-based RT space does not work on the dd_subdomain_oversampled
    # grid view modeling the neighborhood (at least is the underlying grid is a 2d simplex alugrid) and we thus
    # need to reconstruct on the full leaf view.
    restricted_u_h_with_global_support_vector = block_space.project_onto_neighborhood(
        [localized_u_h[ii] if ii in neighborhood else Vector(block_space.local_space(ii).size(), 0.)
         for ii in range(grid.num_subdomains)],
        set([ii for ii in range(grid.num_subdomains)]))
    restricted_u_h_with_global_support = make_discrete_function(block_space,
                                                                restricted_u_h_with_global_support_vector)
    local_ESV2007_residual_product = make_local_ESV2007_residual_product(grid, ii, -1,
                                                                         lambda_, kappa, f,
                                                                         restricted_u_h_with_global_support,
                                                                         restricted_u_h_with_global_support,
                                                                         over_integrate=2)
    eta_r_ESV2007_squared += local_ESV2007_residual_product.apply2()
    local_OS2015_residual_product = make_local_OS2015_residual_product(grid, ii, -1,
                                                                       lambda_, lambda_, kappa, f,
                                                                       restricted_u_h_with_global_support,
                                                                       restricted_u_h_with_global_support,
                                                                       over_integrate=2)
    eta_r_OS2015_squared += local_OS2015_residual_product.apply2()
    # eta df
    # The same applies as above.
    local_diffusive_flux_product = make_local_diffusive_flux_product(grid, ii, -1,
                                                                     lambda_, kappa,
                                                                     restricted_u_h_with_global_support,
                                                                     restricted_u_h_with_global_support,
                                                                     over_integrate=2)
    eta_df_squared += local_diffusive_flux_product.apply2()

print('  nonconformity indicator:      {} (should be 1.66e-01)'.format(np.sqrt(eta_nc_squared)))
print('  residual indicator (ESV2007): {} (should be 7.23e-02)'.format(np.sqrt(eta_r_ESV2007_squared)))
print('  residual indicator (OS2015):  {} (should be 2.89e-01)'.format(np.sqrt(eta_r_OS2015_squared)))
print('  diffusive flux indicator:     {} (should be 3.55e-01)'.format(np.sqrt(eta_df_squared)))